## Embeddings

Primero vamos a ver cómo funciona un word embedding de una forma simulada al tiempo que vemos como emplear la capa de embeddings de Keras, luego veremos como hacer sentence embedding utilizando un modulo o modelo preentrenado de Tensorflow.

In [ ]:
# Importamos las librerías necesarias para trabajar con embeddings

# NumPy: Biblioteca fundamental para computación científica y manejo de arrays
import numpy as np

# TensorFlow: Framework de deep learning que usaremos para crear y entrenar modelos
# También incluye Keras, una API de alto nivel para redes neuronales
import tensorflow as tf

### Word Embeddings

Recuerda que un word embedding transforma las palabras de un texto en un vector de n dimensiones. Veamos como hacerlo con una capa de embeddings, sin entrenar y así podrás ver como instanciarla.

In [ ]:
# ============================================================================
# EJEMPLO DE WORD EMBEDDING CON KERAS
# ============================================================================
# Vamos a crear un vocabulario de ejemplo basado en una frase conocida
# Cada palabra será convertida en un vector de números (embedding)

# Vocabulario de ejemplo: frase de "La princesa prometida"
categorias_ejemplo = ["Me","llamo","Iñigo","Montoya","soy","tú","mataste","a","mi","padre"]

# PASO 1: CONVERTIR PALABRAS A ÍNDICES NUMÉRICOS
# -----------------------------------------------
# StringLookup es una capa de preprocesamiento que asigna un número único a cada palabra
# Esto es necesario porque las redes neuronales trabajan con números, no con texto
pre_conversion = tf.keras.layers.StringLookup()

# adapt() es como hacer un "fit" en sklearn: analiza el vocabulario y crea el mapeo
# Después de esto, cada palabra tendrá asignado un número único
pre_conversion.adapt(categorias_ejemplo)

# PASO 2: CREAR EL MODELO DE LOOKUP + EMBEDDING
# ----------------------------------------------
# Sequential permite encadenar capas de forma secuencial
lookup_y_embedding = tf.keras.Sequential([
    # Capa de entrada: recibe strings (palabras)
    tf.keras.layers.InputLayer(shape=[], dtype=tf.string),
    
    # Capa 1: Convierte palabras en índices numéricos
    pre_conversion,
    
    # Capa 2: Embedding - convierte índices en vectores densos
    tf.keras.layers.Embedding(
        input_dim = pre_conversion.vocabulary_size(),  # Tamaño del vocabulario (cuántas palabras únicas tenemos)
        output_dim = 2  # Dimensión del embedding (convertiremos cada palabra en un vector de 2 números)
    )
])

# NOTA IMPORTANTE: input_dim es el tamaño del vocabulario
#                  output_dim es la dimensión del vector embedding resultante
# En este caso, cada palabra se convertirá en un vector de 2 dimensiones [x, y]

Este "modelo" no resuelve ningún tipo de problema solo pasa las palabras a traves de la capa de codificación y luego de la embeddings y genera por cadda palabra un vector de 2 dimensiones (output_dim). Pero además como no está entrenada funcionará porque tiene pesos inicializados de forma aleatoria. Es decir que si le pasamos como entrada la variable con la frase de ejemplo...

In [ ]:
# VISUALIZACIÓN DEL PROCESO DE CODIFICACIÓN
# ==========================================
# Veamos cómo la capa StringLookup convierte nuestras palabras en índices numéricos
# Esto es el primer paso antes de crear los embeddings

pre_conversion(categorias_ejemplo)

# SALIDA ESPERADA: Un tensor con 10 números (uno por cada palabra)
# Cada número es el índice único asignado a esa palabra en el vocabulario
# Por ejemplo: "Me" -> 9, "llamo" -> 6, "Iñigo" -> 10, etc.

In [ ]:
# APLICANDO EL EMBEDDING COMPLETO (LOOKUP + EMBEDDING)
# =====================================================
# Ahora pasamos nuestras palabras por todo el modelo secuencial:
# 1. InputLayer recibe las palabras como strings
# 2. StringLookup las convierte a índices
# 3. Embedding convierte los índices en vectores de 2 dimensiones

lookup_y_embedding(np.array(categorias_ejemplo))

# SALIDA ESPERADA: Una matriz de forma (10, 2)
# - 10 filas: una por cada palabra
# - 2 columnas: las dos dimensiones del embedding
# 
# Cada palabra ahora está representada por 2 números (ej: [-0.044, 0.034])
# IMPORTANTE: Como la capa Embedding no está entrenada, los valores son ALEATORIOS
# En un modelo real, estos valores se aprenderían durante el entrenamiento
# para capturar el significado semántico de las palabras

Nos convierte cada palabra en un embedding (sin sentido)...

Otra forma de hacerlo

In [ ]:
# FORMA ALTERNATIVA: PROCESANDO UNA FRASE COMPLETA
# =================================================
# En lugar de pasar una lista de palabras, podemos procesar una frase directamente
# usando split() para separar las palabras

frase = "Me llamo Iñigo Montoya"

# split() divide la frase en palabras usando espacios como separador
# ["Me", "llamo", "Iñigo", "Montoya"]
lookup_y_embedding(np.array(frase.split()))

# SALIDA: Matriz de (4, 2) - 4 palabras convertidas en vectores de 2 dimensiones
# Observa que las palabras que aparecían en el vocabulario original
# tendrán los mismos embeddings que antes (porque son las mismas palabras)

Para poder darle valor tendríamos que incluir nuestras dos capas (la codificadora y la de embedding) en un modelo con un objetivo determinado y la capa de embeddings se entrenaría para generar los embeddings que mejor se adapten al problema a solucionar con ese modelo

### Sentences embedding

Vamos a convertir una serie de frases en embeddings. En concreto de 50 dimensiones. Lo haremos utilizando un modelo preentrenado el nnlem-en-dim50 de Google. Internamente es un modelo word embeddings que convierte cada palabra en un embedding de 50 dimensiones y luego calcular el centroide de todos los vectores obtenidos para una frase.

In [ ]:
# ============================================================================
# SENTENCE EMBEDDINGS CON TENSORFLOW HUB
# ============================================================================
# Ahora vamos a usar un modelo PREENTRENADO para convertir frases completas en embeddings
# A diferencia del ejemplo anterior, este modelo ya ha sido entrenado con millones de textos

# Importamos TensorFlow Hub: repositorio de modelos preentrenados
import tensorflow_hub as hub

# CARGANDO EL MODELO PREENTRENADO
# --------------------------------
# Modelo: NNLM (Neural Network Language Model) de Google
# - "en": entrenado en inglés
# - "dim50": genera embeddings de 50 dimensiones
# - "2": versión 2 del modelo
# 
# Este modelo funciona así:
# 1. Toma cada palabra de la frase y la convierte en un embedding de 50 dimensiones
# 2. Calcula el CENTROIDE (promedio) de todos los vectores de las palabras
# 3. El resultado es un único vector de 50 dimensiones que representa toda la frase
hub_layer = hub.KerasLayer("https://tfhub.dev/google/nnlm-en-dim50/2")

# PROBANDO EL MODELO CON DOS FRASES FAMOSAS DE SHAKESPEARE
# ---------------------------------------------------------
# "To be" y "Not to be" son frases del famoso soliloquio de Hamlet
sentence_embeddings = hub_layer(tf.constant(["To be", "Not to be"]))

# Mostramos los embeddings redondeados a 2 decimales para mejor legibilidad
sentence_embeddings.numpy().round(2)

# SALIDA ESPERADA: Matriz de (2, 50)
# - 2 filas: una por cada frase
# - 50 columnas: las 50 dimensiones del embedding
# 
# VENTAJA: Estos embeddings SÍ tienen significado semántico porque el modelo
# fue entrenado con grandes cantidades de texto. Frases similares tendrán
# embeddings cercanos en el espacio vectorial de 50 dimensiones.

Probemos ahora algunas cosas como por ejemplo obtener la similitud entre sentencias

In [ ]:
# CORPUS DE EJEMPLO: FRASES EN ESPAÑOL SOBRE DIFERENTES TEMAS
# =============================================================
# Vamos a probar el modelo con frases en español (aunque fue entrenado en inglés)
# Tenemos dos categorías de frases:
# - Frases 1 y 2: Sobre fútbol (Real Madrid y Barcelona)
# - Frases 3 y 4: Sobre la guerra Rusia-Ucrania

sentences = ['El Real Madrid lo tiene difícil para ganar al Manchester City.',
             'El Barcelona puede clasificar frente al PSG, si se esfuerza.',
             'Las tropas rusas han tomado Dubroknic.',
             'El ejercito ucraniano se ha replegado']

# OBJETIVO: Verificar si el modelo puede capturar la similitud semántica
# incluso en español (idioma diferente al de entrenamiento)
# - Las frases 1-2 deberían ser similares entre sí (ambas de fútbol)
# - Las frases 3-4 deberían ser similares entre sí (ambas de guerra)
# - Las frases de diferentes temas deberían ser menos similares

In [ ]:
# GENERANDO EMBEDDINGS PARA NUESTRAS 4 FRASES EN ESPAÑOL
# ========================================================
# Convertimos cada frase en un vector de 50 dimensiones

sentence_embeddings = hub_layer(tf.constant(sentences))

# RESULTADO: Matriz de (4, 50)
# - 4 frases → 4 vectores
# - Cada vector tiene 50 dimensiones
# 
# Ahora cada frase está representada en un espacio vectorial donde:
# - Frases con significados similares estarán CERCA
# - Frases con significados diferentes estarán LEJOS

In [ ]:
# ============================================================================
# CALCULANDO SIMILITUD ENTRE TODAS LAS FRASES
# ============================================================================
# Vamos a comparar cada frase con todas las demás usando dos métricas:
# 1. SIMILITUD DE COSENO: mide el ángulo entre vectores (valores entre -1 y 1)
# 2. DISTANCIA EUCLIDIANA: mide la distancia "directa" entre vectores

# Importamos las herramientas necesarias
from sklearn.metrics.pairwise import cosine_similarity  # Para calcular similitud de coseno
from itertools import combinations  # Para generar todas las combinaciones posibles

# COMPARACIÓN DE TODAS LAS COMBINACIONES POSIBLES
# -----------------------------------------------
# combinations(lista, r=2) genera todos los pares posibles sin repetir
# Por ejemplo: (1,2), (1,3), (1,4), (2,3), (2,4), (3,4)

for (frase1, vec1), (frase2, vec2) in combinations(zip(sentences, sentence_embeddings.numpy()), r=2):
    # Calculamos dos métricas de similitud:
    
    # 1. SIMILITUD DE COSENO (valores cercanos a 1 = muy similares)
    similitud_coseno = cosine_similarity([vec1], [vec2])
    
    # 2. DISTANCIA EUCLIDIANA (valores pequeños = muy similares)
    distancia_euclidiana = np.linalg.norm(vec1 - vec2)
    
    # Mostramos los resultados
    print(frase1, "vs", frase2, similitud_coseno, distancia_euclidiana)

# INTERPRETACIÓN DE RESULTADOS:
# -----------------------------
# SIMILITUD DE COSENO:
# - Valores cercanos a 1: frases muy similares
# - Valores cercanos a 0: frases no relacionadas
# - Valores cercanos a -1: frases opuestas
#
# DISTANCIA EUCLIDIANA:
# - Valores pequeños: frases similares
# - Valores grandes: frases diferentes
#
# ESPERADO: Las frases de fútbol (1-2) y las de guerra (3-4) deberían
# tener mayor similitud entre sí que frases de temas diferentes

Con el coseno tendríamos algún problema con la distancia quedan mejor emparejadas.

In [ ]:
# ============================================================================
# SISTEMA DE PREGUNTAS Y RESPUESTAS CON EMBEDDINGS
# ============================================================================
# Vamos a crear un sistema básico de Q&A que encuentra la respuesta más relevante
# para una pregunta dada, basándose en la similitud de embeddings

# PREGUNTA 1: Sobre el Barcelona
# --------------------------------
question = "¿Contra quién juega el Barcelona?"

# Paso 1: Convertir la pregunta en un embedding de 50 dimensiones
pregunta = hub_layer(tf.constant([question]))
vec_q = pregunta.numpy()  # Convertimos a numpy array para facilitar cálculos

# Paso 2: Calcular distancia entre la pregunta y cada frase candidata
distancias = []  # Almacenará las distancias
respuestas = []  # Almacenará las frases candidatas

for answer, vec_a in zip(sentences, sentence_embeddings.numpy()):
    respuestas.append(answer)
    
    # Calculamos distancia euclidiana entre embedding de pregunta y respuesta
    # Menor distancia = mayor similitud semántica
    distancias.append(np.linalg.norm(vec_q - vec_a))

# Paso 3: Encontrar la respuesta con menor distancia (más similar)
# np.argmin() devuelve el índice del valor mínimo
indice_mejor_respuesta = np.argmin(distancias)

# Mostramos pregunta y respuesta
print(f"P:{question}")
print(f"R:{respuestas[indice_mejor_respuesta]}")

# FUNCIONAMIENTO:
# ---------------
# El modelo encuentra que "El Barcelona puede clasificar frente al PSG..."
# es la frase más similar a la pregunta porque:
# - Ambas contienen la palabra "Barcelona"
# - Ambas tienen contexto de fútbol
# - Los embeddings capturan esta similitud semántica

In [ ]:
# PREGUNTA 2: Sobre Ucrania
# --------------------------
# Probamos el mismo sistema con una pregunta sobre tema militar

question = "¿Qué hacen los ucranianos?"

# Convertimos la pregunta en embedding
pregunta = hub_layer(tf.constant([question]))
vec_q = pregunta.numpy()

# Calculamos distancias con todas las frases candidatas
distancias = []
respuestas = []

for answer, vec_a in zip(sentences, sentence_embeddings.numpy()):
    respuestas.append(answer)
    distancias.append(np.linalg.norm(vec_q - vec_a))

# Encontramos la frase más similar
print(f"P:{question}")
print(f"R:{respuestas[np.argmin(distancias)]}")

# RESULTADO ESPERADO:
# -------------------
# Debería devolver "El ejercito ucraniano se ha replegado"
# porque es la única frase que menciona acciones de los ucranianos
# 
# NOTA: El embedding captura que "ucranianos" y "ucraniano" son similares
# incluso con diferente forma (plural vs singular)

In [ ]:
# PREGUNTA 3: Sobre una ubicación específica
# -------------------------------------------
# Probamos con una pregunta sobre un lugar específico (Dubrovnik)

question = "¿Qué han pasado en Dubrocnick?"

# Convertimos la pregunta en embedding
pregunta = hub_layer(tf.constant([question]))
vec_q = pregunta.numpy()

# Calculamos distancias con todas las frases candidatas
distancias = []
respuestas = []

for answer, vec_a in zip(sentences, sentence_embeddings.numpy()):
    respuestas.append(answer)
    distancias.append(np.linalg.norm(vec_q - vec_a))

# Encontramos la frase más similar
print(f"P:{question}")
print(f"R:{respuestas[np.argmin(distancias)]}")

# RESULTADO ESPERADO:
# -------------------
# Debería devolver "Las tropas rusas han tomado Dubroknic."
# 
# ASPECTO INTERESANTE:
# --------------------
# Observa que en la pregunta escribimos "Dubrocnick" (con error ortográfico)
# pero el sistema encuentra la respuesta correcta con "Dubroknic"
# Esto demuestra la ROBUSTEZ de los embeddings ante pequeñas variaciones
# en la escritura, porque capturan el significado semántico general
# 
# APLICACIONES PRÁCTICAS:
# -----------------------
# Este tipo de sistema se usa en:
# - Chatbots que buscan respuestas en una base de conocimiento
# - Buscadores semánticos que encuentran documentos relevantes
# - Sistemas de recomendación de contenido similar
# - FAQ automáticos que encuentran la pregunta frecuente más parecida